In [ ]:
# Abrindo no encoding original (latin-1 ou cp1252)
with open("./2025-06-06_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv", "r") as f:
    conteudo = f.read()

# Salvando em UTF-8
with open("base-utf8.csv", "w", encoding="utf-8") as f:
    f.write(conteudo)

In [1]:
import pandas as pd

df = pd.read_csv("./base-utf8.csv", sep=";", encoding="utf-8", usecols=["Data do Atendimento", "Sexo", "Tipo de Unidade", "Descrição do Procedimento", "Descrição do CBO", "Nacionalidade"])

In [2]:
print(df["Sexo"].value_counts())
print(df["Tipo de Unidade"].value_counts())

Sexo
F    761552
M    456842
Name: count, dtype: int64
Tipo de Unidade
BASICO    815385
UPA       368363
SIACE      34646
Name: count, dtype: int64


In [3]:
df_upa = df[df["Tipo de Unidade"] == "UPA"].copy()
print(df_upa["Tipo de Unidade"].value_counts())

Tipo de Unidade
UPA    368363
Name: count, dtype: int64


In [4]:
(df_upa.isnull().sum()).sort_values(ascending=False)

Data do Atendimento          0
Sexo                         0
Tipo de Unidade              0
Descrição do Procedimento    0
Descrição do CBO             0
Nacionalidade                0
dtype: int64

In [5]:
print(df_upa["Descrição do Procedimento"].value_counts())

df_upa["Descrição do Procedimento"] = (
    df_upa["Descrição do Procedimento"]
    .str.upper()                       
    .str.normalize("NFKD")            
    .str.encode("ascii", errors="ignore") 
    .str.decode("utf-8")
    .str.replace(r"[^A-Z0-9 ]", "", regex=True)  
    .str.strip()                       
)

print(df_upa["Descrição do Procedimento"].value_counts())


Descrição do Procedimento
ATENDIMENTO MEDICO EM UNIDADE DE PRONTO ATENDIMENTO                            277378
ATENDIMENTO DE URGENCIA C/ OBSERVACAO ATE 24 HORAS EM ATENCAO ESPECIALIZADA     90922
ACOLHIMENTO COM CLASSIFICACAO DE RISCO                                             63
Name: count, dtype: int64
Descrição do Procedimento
ATENDIMENTO MEDICO EM UNIDADE DE PRONTO ATENDIMENTO                           277378
ATENDIMENTO DE URGENCIA C OBSERVACAO ATE 24 HORAS EM ATENCAO ESPECIALIZADA     90922
ACOLHIMENTO COM CLASSIFICACAO DE RISCO                                            63
Name: count, dtype: int64


In [6]:
mapa_categoria = {
    'ATENDIMENTO MEDICO EM UNIDADE DE PRONTO ATENDIMENTO': 'PRONTO ATENDIMENTO',
    'ATENDIMENTO DE URGENCIA C OBSERVACAO ATE 24 HORAS EM ATENCAO ESPECIALIZADA': 'URGENCIA/OBSERVACAO',
    'ACOLHIMENTO COM CLASSIFICACAO DE RISCO': 'ACOLHIMENTO'
}

df_upa['Tipo Procedimento'] = df_upa['Descrição do Procedimento'].map(mapa_categoria)

In [7]:
df_upa["data_convertida"] = pd.to_datetime(df_upa["Data do Atendimento"], errors="coerce")
df_upa["Data Iso"] = df_upa["data_convertida"].dt.strftime("%Y-%m-%dT%H:%M:%S")
df_upa = df_upa.drop(columns=["data_convertida"])

In [8]:
print(df_upa["Nacionalidade"].value_counts())

df_upa["Nacionalidade"] = (
    df_upa["Nacionalidade"]
    .str.upper()                       
    .str.normalize("NFKD")            
    .str.encode("ascii", errors="ignore") 
    .str.decode("utf-8")
    .str.replace(r"[^A-Z0-9 ]", "", regex=True)  
    .str.strip()                       
)

print(df_upa["Nacionalidade"].value_counts())


Nacionalidade
Brasileira       342832
Estrangeiro       16490
Não informado      9006
Naturalizado         35
Name: count, dtype: int64
Nacionalidade
BRASILEIRA       342832
ESTRANGEIRO       16490
NAO INFORMADO      9006
NATURALIZADO         35
Name: count, dtype: int64


In [9]:
print(df_upa["Descrição do CBO"].value_counts())

df_upa["Descrição do CBO"] = (
    df_upa["Descrição do CBO"]
    .str.upper()                       
    .str.normalize("NFKD")            
    .str.encode("ascii", errors="ignore") 
    .str.decode("utf-8")
    .str.replace(r"[^A-Z0-9 ]", "", regex=True)  
    .str.strip()                       
)

print(df_upa["Descrição do CBO"].value_counts())


Descrição do CBO
MEDICO CLINICO        351365
MEDICO PEDIATRA        16041
MEDICO GENERALISTA       920
MEDICO RESIDENTE          37
Name: count, dtype: int64
Descrição do CBO
MEDICO CLINICO        351365
MEDICO PEDIATRA        16041
MEDICO GENERALISTA       920
MEDICO RESIDENTE          37
Name: count, dtype: int64


In [10]:
import unicodedata

def padronizar_coluna(col):
    # Remove acentos
    col = ''.join(
        c for c in unicodedata.normalize('NFKD', col)
        if not unicodedata.combining(c)
    )
    # Remove caracteres especiais que não sejam letras, números ou espaço
    col = ''.join(c for c in col if c.isalnum() or c == ' ')
    # Converte para maiúsculo
    col = col.upper()
    # Substitui espaços por _
    col = col.replace(' ', '_')
    return col

df_upa.columns = [padronizar_coluna(c) for c in df_upa.columns]

In [11]:
df_upa.to_csv("output-atendimentos-limpo.csv", index=False)